In [25]:
import gym
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [69]:
M = 20  # kg\n",
g = 9.8 # m / s^2\n",
m = 0.5 # kg\n",
l = 0.5 # m"
v1 = ((M + m) * g) / (M * l)
v2 = -(m * g) / M
c1 = -1 / (M * l)
c2 = 1 / M
global A
global B
A = np.array([[0, 1, 0, 0], [0, 0, v2, 0], [0, 0, 0, 1], [0, 0, v1, 0]])
B = np.array([[0], [c2], [0], [c1]])

In [70]:
def invertedpendulum(state,action):
  state=np.array(state)
  action=np.array(action)
  state_gradient=np.dot(A,state.reshape(4,1))+np.dot(B,action-110)
  state1=state+1e-2*state_gradient.reshape(4)
  done=False
  if(abs(state1[2])<np.pi/12):
    reward=5
  elif(abs(state1[2])<np.pi/10):
    reward=4
  elif(abs(state1[2])<np.pi/8):
    reward=3
  elif(abs(state1[2])<np.pi/6):
    reward=2
  elif(abs(state1[2])<np.pi/4):
    reward=1
  else:
    reward=0
    done=True
  # print(state,action-100,state1)
  return state1,reward,done


In [71]:
# DEVICE="cuda:0"
ACTION_SPACE = np.arange(0,300)
# ACTION_SPACE = [0,1]
print(ACTION_SPACE)
EPISODES = 30000
STEPS = 1000
GAMMA=1
RENDER=False
class ReinforceModel(nn.Module):
    def __init__(self,num_action,num_input):
        super(ReinforceModel,self).__init__()
        self.num_action = num_action
        self.num_input = num_input

        self.layer1 = nn.Linear(num_input,64)
        self.layer2 = nn.Linear(64,num_action)
        
    def forward(self,x):
        x = torch.tensor(x,dtype=torch.float32).unsqueeze(0)
        x = F.relu(self.layer1(x))
        x=x.detach().numpy()
        x = np.round(x, decimals=7)
        x=torch.tensor(x)
        actions = F.softmax(self.layer2(x))
        try:
          action = self.get_action(actions)
        except:
          print(x)
          print(self.layer2.weight)
          print(self.layer2(x))
          actions=actions.detach().numpy()
          actions[np.isnan(actions)] = 0
          actions /= actions.sum()
          print(actions)
          actions=torch.tensor(actions)
          action = self.get_action(actions)
        a=torch.log(actions.squeeze(0))
        log_prob_action = a[action]
        return action,log_prob_action
    def get_action(self,a):
        return np.random.choice(ACTION_SPACE,p=a.squeeze(0).detach().cpu().numpy())
    
model = ReinforceModel(300,4)
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
all_rewards =[]
for episode in range(EPISODES):
    done=False
 
    state = [0,0,np.pi/6,0]
    lp=[]
    a=[]
    r=[]
    d=[]
    s=[]

    for step in range(STEPS):
        action,log_prob = model(state)
        # print(action)
        # print(log_prob)
        state,r_,done = invertedpendulum(state,action)
        lp.append(log_prob)
        r.append(r_)
        if done:
            all_rewards.append(np.sum(r))
            if episode%100 ==0:
                print(f"EPISODE {episode} SCORE: {np.sum(r)} roll{pd.Series(all_rewards).tail(30).mean()}")
            
            break
    discounted_rewards = np.zeros_like(r)

    discounted_rewards = []

    for t in range(len(r)):
        Gt = 0 
        pw = 0
        for r_ in r[t:]:
            Gt = Gt + GAMMA**pw * r_
            pw = pw + 1
        discounted_rewards.append(Gt)
    
    discounted_rewards = np.array(discounted_rewards)
        

          


    discounted_rewards = torch.tensor(discounted_rewards,dtype=torch.float32)
    discounted_rewards = (discounted_rewards - torch.mean(discounted_rewards))/ (torch.std(discounted_rewards))
    log_prob = torch.stack(lp)
    policy_gradient = -log_prob*discounted_rewards

    model.zero_grad()
    policy_gradient.sum().backward()
    optimizer.step()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-71-233df94f3868>:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  actions = F.softmax(self.layer2(x))


[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 244 245 24

KeyboardInterrupt: ignored

In [67]:
rewards=[]
state = [0,0,np.pi/6,0]
done=False
while(done==False):
    action,log_prob = model(state)
    print(state)
    state,reward,done = invertedpendulum(state,action)
    rewards.append(reward)

<ipython-input-66-c6ec04f81e10>:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  actions = F.softmax(self.layer2(x))


[0, 0, 0.5235987755982988, 0]
[ 0.          0.11371718  0.52359878 -0.12480901]
[0.00113717 0.08743437 0.52235069 0.03038199]
[0.00201152 0.06115461 0.52265451 0.18532224]
[0.00262306 0.1748741  0.52450773 0.06032353]
[0.0043718  0.14858906 0.52511096 0.21569713]
[0.00585769 0.12230254 0.52726793 0.37119193]
[0.00708072 0.23601073 0.53097985 0.24712005]
[0.00944083 0.20970983 0.53345105 0.40379391]
[0.01153792 0.32340288 0.53748899 0.28096422]
[0.01477195 0.29708603 0.54029864 0.43894576]
[0.01774281 0.4107623  0.54468809 0.31749176]
[0.02185044 0.53342781 0.54786301 0.1789196 ]
[0.02718471 0.65608555 0.54965221 0.04098527]
[0.03374557 0.6297389  0.55006206 0.2014104 ]
[0.04004296 0.75239125 0.55207616 0.06391787]
[ 0.04756687  0.86603866  0.55271534 -0.05517003]
[ 0.05622726  0.97968451  0.55216364 -0.17412952]
[ 0.0660241   0.95333171  0.55042235 -0.01319984]
[0.07555742 0.92698317 0.55029035 0.14738001]
[0.08482725 0.90063496 0.55176415 0.30793334]
[0.0938336  1.00678314 0.55484348 

In [ ]:
print(rewards,len(rewards))

In [1]:
# actions=[1,2,3,4,5]
# print(min(actions),max(actions))

1 5


In [15]:
A=[]
x=1
for i in range(1000):
  A.append(x/10.0)
  x=x/10.0
A=np.array(A)
A[np.isnan(A)] = 0
print(A)

[1.000000e-001 1.000000e-002 1.000000e-003 1.000000e-004 1.000000e-005
 1.000000e-006 1.000000e-007 1.000000e-008 1.000000e-009 1.000000e-010
 1.000000e-011 1.000000e-012 1.000000e-013 1.000000e-014 1.000000e-015
 1.000000e-016 1.000000e-017 1.000000e-018 1.000000e-019 1.000000e-020
 1.000000e-021 1.000000e-022 1.000000e-023 1.000000e-024 1.000000e-025
 1.000000e-026 1.000000e-027 1.000000e-028 1.000000e-029 1.000000e-030
 1.000000e-031 1.000000e-032 1.000000e-033 1.000000e-034 1.000000e-035
 1.000000e-036 1.000000e-037 1.000000e-038 1.000000e-039 1.000000e-040
 1.000000e-041 1.000000e-042 1.000000e-043 1.000000e-044 1.000000e-045
 1.000000e-046 1.000000e-047 1.000000e-048 1.000000e-049 1.000000e-050
 1.000000e-051 1.000000e-052 1.000000e-053 1.000000e-054 1.000000e-055
 1.000000e-056 1.000000e-057 1.000000e-058 1.000000e-059 1.000000e-060
 1.000000e-061 1.000000e-062 1.000000e-063 1.000000e-064 1.000000e-065
 1.000000e-066 1.000000e-067 1.000000e-068 1.000000e-069 1.000000e-070
 1.000